# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()

In [6]:
#Obetenemos el codigo HTML

URL = "https://ycharts.com/companies/TSLA/revenues"

driver.get(URL)

In [34]:
#Obtenemos el TAG
raw_data = driver.find_elements(By.TAG_NAME, "tr")

#Filtramos los datos que nos interesan

data = []
for i in raw_data:
    data.append(i.text)
    if ("General Motors Co" in str(i.text)):
        data.remove(i.text)
        break

date = []
revenue = []
for i in range(len(data)):
    revenue.append (data[i].split(" ")[-1])

revenue



['',
 '',
 '',
 '',
 '',
 '',
 '',
 'Value',
 '25.50B',
 '21.30B',
 '25.17B',
 '23.35B',
 '24.93B',
 '23.33B',
 '24.32B',
 '21.45B',
 '16.93B',
 '18.76B',
 '17.72B',
 '13.76B',
 '11.96B',
 '10.39B',
 '10.74B',
 '8.771B',
 '6.036B',
 '5.985B',
 '7.384B',
 '6.303B',
 '6.35B',
 '4.541B',
 '7.226B',
 '6.824B',
 '4.002B',
 'Value',
 '3.409B',
 '3.288B',
 '2.985B',
 '2.790B',
 '2.696B',
 '2.285B',
 '2.298B',
 '1.270B',
 '1.147B',
 '1.214B',
 '936.79M',
 '954.98M',
 '939.88M',
 '956.66M',
 '851.80M',
 '769.35M',
 '620.54M',
 '615.22M',
 '431.35M',
 '405.14M',
 '561.79M',
 '306.33M',
 '50.10M',
 '26.65M',
 '30.17M']

In [3]:
driver.quit()